<a href="https://colab.research.google.com/github/100milliongold/Gemma_3_Fine_tuning/blob/main/Gemma_3_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up

원본 출처 https://www.kaggle.com/code/kingabzpro/fine-tuning-gemma-3-finq-a-reasoning

KorQuAD/squad_kor_v1 데이터셋을 파인 튜닝하도록 수정함.

by webnautes

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [ ]:
%%capture
%pip install -U datasets
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U bitsandbytes

In [ ]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')
login(hf_token)

## Loading the model and tokenizer

In [ ]:
from transformers import AutoTokenizer, Gemma3ForConditionalGeneration
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)

import torch


model = Gemma3ForConditionalGeneration.from_pretrained(
    "google/gemma-3-4b-it",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager'
).eval()

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

## Loading and processing the dataset

In [ ]:
train_prompt_style="""
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Question:
{}

### Response:
{}
"""

In [ ]:
def formatting_prompts_func(examples):
    inputs = examples["question"]
    outputs = examples["answers"]
    texts = []
    for question, response in zip(inputs, outputs):
        # print(question)
        response  = response["text"][0]
        # print(response)
        # Append the EOS token to the response if it's not already there
        if not response.endswith(tokenizer.eos_token):
            response += tokenizer.eos_token
        text = train_prompt_style.format(question,  response)
        texts.append(text)
    return {"text": texts}

In [ ]:
from datasets import load_dataset
dataset = load_dataset("KorQuAD/squad_kor_v1", split = "train[0:500]",trust_remote_code=True)
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

README.md:   0%|          | 0.00/6.29k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

'\nBelow is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Question:\n바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?\n\n### Response:\n교향곡<eos>\n'

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

## Model inference before fine-tuning

In [ ]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Question:
{}

### Response:
"""

In [ ]:
question = dataset[0]['question']

print(question)

inputs = tokenizer(
    [prompt_style.format(question, "") + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

# 1. timeout 설정 추가
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=500,  # 줄여보기
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
    do_sample=True,  # 확률적 샘플링 활성화
    temperature=0.7,  # 약간의 무작위성 추가
    num_return_sequences=1,
    no_repeat_ngram_size=3,  # 반복 방지
)

# 2. 다른 방법: 더 작은 텍스트 청크로 나누어 생성
# 3. 런타임 모니터링: 생성 중 메모리 사용량과 GPU 활용도 확인
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response[0].split("### Response:")[1])

바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


바 그너는 헤르만 헤세의 도움을 받아 괴테 파우즈트를 바탕으로 '바그 너'를 썼습니다. 바그너에게 파우스터는 영웅적인 존재가 아닌 악마였고, 인간의 욕망을 쫓는 존재였습니다. 바 그너의 '바 그 너'는 파우저의 이야기를 재해석하여 인간의 어두운 면을 드러내고자 했습니다.


Richard Wagner wrote 'Wagner' based on Goethe's Faust with the help of Hermann Hesse. Wagner saw Faust not as an heroic figure but as a devil, a being pursuing human desires. Wagner's 'Wagner’ aimed to reveal the darker aspects of human nature by reinterpreting Faust's story.

Show answer
..
.Show answer.Answer:
Richard Wager wrote "Wagner" based on the help from Hermann Hesse, using Goethe’s “Faust” as a foundation. Wagner viewed Faust not merely as a heroic figure, but as an embodiment of demonic desires relentlessly pursuing human ambition. Consequently, “Wagner” sought to expose the darker facets of human existence by re-imagining the narrative of Faust.Show answer.
Answer:

Richard Wagner, with the assistance of Hermann Hess, wrote "The Master Builder" (Der Meisterbuilder) inspired by Goethe’ss "F

## Setting up the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# LoRA Configuration
peft_config = LoraConfig(
    lora_alpha=8,                           # Scaling factor for LoRA
    lora_dropout=0.05,                       # Add slight dropout for regularization
    r=64,                                    # Rank of the LoRA update matrices
    bias="none",                             # No bias reparameterization
    task_type="CAUSAL_LM",                   # Task type: Causal Language Modeling
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # Target modules for LoRA
)


# Training Arguments
training_arguments = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=3,
    logging_steps=0.2,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="none"
)

# Initialize the Trainer
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset,
    peft_config=peft_config,
    data_collator=data_collator,
)

Converting train dataset to ChatML:   0%|          | 0/500 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Model training

In [ ]:
torch.cuda.empty_cache()
trainer_stats = trainer.train()

Step,Training Loss
150,2.179100
300,1.266900
450,0.959200
600,0.746200
750,0.594300


## Model inference after fine-tuning

In [ ]:
# 옵션 1: 훈련된 모델을 명시적으로 로드
trained_model = trainer.model

# 또는 옵션 2: 훈련된 모델 저장 후 다시 로드
# trainer.save_model("output/final_model")
# trained_model = PeftModel.from_pretrained(model, "output/final_model")

# 모델을 추론 모드로 설정
trained_model.eval()

# 생성 설정 업데이트
question = dataset[0]['question']

inputs = tokenizer(
    [prompt_style.format(question, "")],
    return_tensors="pt"
).to("cuda")

# 생성 파라미터 업데이트 및 안전 조치 추가
outputs = trained_model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
    do_sample=True,  # 더 다양한 출력을 위해
    temperature=0.7,  # 창의성 조절
    top_p=0.9,  # 상위 확률 토큰으로 제한
)

# 응답 처리 개선
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print("전체 응답:", response[0])  # 전체 응답 확인

# 응답 파싱 시도
try:
    parsed_response = response[0].split("### Response:")[1].strip()
    print("파싱된 응답:", parsed_response)
except IndexError:
    print("'### Response:' 형식을 찾을 수 없습니다. 응답 형식을 확인하세요.")
    # 다른 분리자 시도
    if "Response:" in response[0]:
        parsed_response = response[0].split("Response:")[1].strip()
        print("대체 파싱된 응답:", parsed_response)

전체 응답: Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Question:
바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?

### Response:
교향곡
파싱된 응답: 교향곡


In [ ]:
# 생성 설정 업데이트
question = dataset[10]['question']

inputs = tokenizer(
    [prompt_style.format(question, "")],
    return_tensors="pt"
).to("cuda")

# 생성 파라미터 업데이트 및 안전 조치 추가
outputs = trained_model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
    do_sample=True,  # 더 다양한 출력을 위해
    temperature=0.7,  # 창의성 조절
    top_p=0.9,  # 상위 확률 토큰으로 제한
)

# 응답 처리 개선
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print("전체 응답:", response[0])  # 전체 응답 확인

# 응답 파싱 시도
try:
    parsed_response = response[0].split("### Response:")[1].strip()
    print("파싱된 응답:", parsed_response)
except IndexError:
    print("'### Response:' 형식을 찾을 수 없습니다. 응답 형식을 확인하세요.")
    # 다른 분리자 시도
    if "Response:" in response[0]:
        parsed_response = response[0].split("Response:")[1].strip()
        print("대체 파싱된 응답:", parsed_response)

전체 응답: Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Question:
바그너는 다시 개정된 총보를 얼마를 받고 팔았는가?

### Response:
10루이의 금
파싱된 응답: 10루이의 금


## Saving the model locally

In [ ]:
new_model_local = "Gemma-3-4B-Fin-QA-Reasoning-squad_kor_v1"

model.save_pretrained(new_model_local) # Local saving
tokenizer.save_pretrained(new_model_local)

('Gemma-3-4B-Fin-QA-Reasoning-squad_kor_v1/tokenizer_config.json',
 'Gemma-3-4B-Fin-QA-Reasoning-squad_kor_v1/special_tokens_map.json',
 'Gemma-3-4B-Fin-QA-Reasoning-squad_kor_v1/tokenizer.model',
 'Gemma-3-4B-Fin-QA-Reasoning-squad_kor_v1/added_tokens.json',
 'Gemma-3-4B-Fin-QA-Reasoning-squad_kor_v1/tokenizer.json')